In [2]:
from simulation_functions import index, simulation_, plot
from interpolation_functions import T_K
import numpy as np
import matplotlib.pyplot as plt


In [2]:
t_drive_new = np.loadtxt('t_drive_new.txt') # true time sample. 
ind = index(15,t_drive_new) # 15 means that the time delay is chosen randomly between 0 and 15 days

In [3]:
# getting the indices of each time shifter curve 
index0 = ind[0]
index1 = ind[1]
index2 = ind[2]
index3 = ind[3]
sample = np.sort(ind[5])

#getting the time sample of each curve
ts_0 = sample[index0]
ts_1 = sample[index1]
ts_2 = sample[index2]
ts_3 = sample[index3]
ts = np.array([ts_0,ts_1,ts_2,ts_3])

# finding the magnitudes corresponding to each time shifted sample
f = simulation_(sample) #running simulation
fs_0 = f[index0]
fs_1 = f[index1]
fs_2 = f[index2]
fs_3 = f[index3]

# desired sampling for each curve
tk_0 = T_K(8,ts_0)[0]
tk_1 = T_K(8,ts_1)[0]
tk_2 = T_K(8,ts_2)[0]
tk_3 = T_K(8,ts_3)[0]
tk = np.array([tk_0,tk_1,tk_2,tk_3])

h=8
time_delays = ind[6] 
print(len(ts_0))

1369


In [4]:
# sinc matrix  # make this into for loop
a = np.array([(ts_0[:,np.newaxis]+time_delays[0])-tk_0[np.newaxis,:]])
b = np.array([(ts_1[:,np.newaxis]+time_delays[1])-tk_1[np.newaxis,:]])
c = np.array([(ts_2[:,np.newaxis]+time_delays[2])-tk_2[np.newaxis,:]])
d = np.array([(ts_3[:,np.newaxis]+time_delays[3])-tk_3[np.newaxis,:]])
sinc_matrix = np.sinc(np.concatenate([a,b,c,d],axis =0)/h)
sinc_matrix.shape # 4 x s x k

(4, 1369, 578)

In [5]:
B= np.concatenate([fs_0[np.newaxis,:],fs_1[np.newaxis,:],fs_2[np.newaxis,:],fs_3[np.newaxis,:]])
# B.shape # 4 x s

In [35]:
X = np.zeros(len(tk_0))#:,np.newaxis]
np.dot((sinc_matrix.T),(B-sinc_matrix@X)).shape


MemoryError: 

In [6]:
# failed attempts to generalize linorm to higher dimensions yielded this:
def linorm(S, nit):
    x0 = np.random.rand(S.shape[1])  # chose [1] per the implementation of the power iteration method in wiki
    x0 = x0 /np.linalg.norm(x0)
    for i in range(nit):
        x = np.dot(x0,S)
        xn = np.linalg.norm(x)
        xp = x / xn
        y = np.dot(xp,S.T) # this was np.dot(xp,S.T) before but the dimensions dont match unless i flip it 
        yn = np.linalg.norm(y)  
        if yn < np.dot(x0.T,y)# doesnt work---- right hand side is now a matrix, being compared to a single number yn ...
           break
        x0 = y / yn
    return 1./xn

In [7]:
# l = linorm(sinc_matrix, 21)
# l

In [10]:
def solution_gd(i): 
    '''
    Solution of matrix equation using gradient descent
    input:
    i : the number of shifted curve desired to solve for, options are 0,1,2,3
    output: 
    plots of the solution
    '''
    A = sinc_matrix
    Y = B
    #edit linorm to take 3D array
#     def linorm(S, nit):
#         n1, n2 = np.shape(S)
#         x0 = np.random.rand(1, n1)
#         x0 = x0 / np.sqrt(np.sum(x0 ** 2))    
#         for i in range(nit):
#             x = np.dot(x0, S)
#             xn = np.sqrt(np.sum(x ** 2))
#             xp = x / xn
#             y = np.dot(xp, S.T)
#             yn = np.sqrt(np.sum(y ** 2))       
#             if yn < np.dot(y, x0.T):
#                 break
#             x0 = y / yn
#         return 1./xn

#     mu = linorm(A, 20)/100.
    mu = 0.0038 # just to move on

    X = np.zeros(len(tk_0))

    count = 0
    R = [np.sum(Y**2)]
    epsilon = 0.3

    while (R[-1] > epsilon) and (count < 10000): # calculated this norm^2 for the least squares sol it gave 0.5 so I chose 0.5>0.3
        X_new = X + mu*((A.T)@(Y-A@X))
        X = X_new.copy()
        R.append(np.sum((Y-A@X)**2.))
        count+=1
        
    plt.title('Convergence')
    plt.plot(np.array(R[1:]))
    plt.xlabel('iterations')
    plt.ylabel('error')
    plt.show()
    
    cut = np.logical_and(X < np.max(Y), X > np.min(Y))
    plt.plot(ts[i],Y,'o',label = 'original')
    plt.plot(tk[i][cut],X[cut],'o',label='interpolated', color = 'orange')
    plt.xlabel('time[days]')
    plt.ylabel('magnitude')
    plt.title('sinc interpolated values')
    plt.legend()
    plt.show()

In [11]:
solution_gd(0) # replace 3 with 0,1,2 for other curves

MemoryError: 

In [1]:
# make solution_gd take all the matrix at the same time by editing --> lipshitz function generalization? memory error?
# interpolate on real data or make simulated data more real by adding a gaussian light curve to it
# account for microlensing?

In [3]:
np.mean([1,2])

1.5